In [ ]:
#import libraries

import pandas as pd
import requests
import credentials
import time

from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
key = credentials.youtube

In [ ]:
sentiment_analyzer = SentimentIntensityAnalyzer()

In [ ]:
data_f = pd.read_csv('c:/Users/Diego/Documents/GitHub/youtube-comment-sentiment/Channel Data/ludwig_data_stats.csv')

In [ ]:
sample_f = data_f.head(50)

In [ ]:
def get_values(comment) :
    senti = sentiment_analyzer.polarity_scores(comment)

    pos = senti['pos']
    neu = senti['neu']
    neg = senti['neg']

    return pos, neu, neg

In [ ]:
def get_average(df) :
    main_df = df.groupby(['video_id']).mean().round(2)

    return main_df

In [ ]:
def get_sentiment(df) :

    for index, row in sample_f.iterrows():
        video_id = row['video_id']

        url_comments = f'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=1000&videoId={video_id}&key={key}&order=relevance'
        comment_api = requests.get(url_comments).json()
    
        for comments in comment_api['items'] :
            #video_id = 'k0f4KyujyVw'

            comment = comments['snippet']['topLevelComment']['snippet']['textDisplay']

            pos, neu, neg = get_values(comment)

            temp_df = pd.DataFrame({
                'video_id' : [video_id],
                'pos' : [pos],
                'neu' : [neu],
                'neg' : [neg]})

            df = pd.concat([df, temp_df], ignore_index=True)

            avg_df = get_average(df)

    return avg_df

In [ ]:
df = pd.DataFrame(columns=['video_id','pos','neu','neg'])

main_df = get_sentiment(df)

In [ ]:
main_df

In [ ]:
main_df[['pos','neu','neg']] = main_df[['pos','neu','neg']].applymap(lambda x: "{:.2%}".format(x))

In [ ]:
main_df